In [1]:
! pip install tensorflow scikit-learn pandas numpy pickle5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pickle5: filename=pickle5-0.0.11-cp310-cp310-linux_x86_64.whl size=255318 sha256=074a82249d9895c06cce0d2f5cfb0ea356c074704eb332a3d635f547ef7117c2
  Stored in directory: /root/.cache/pip/wheels/7d/14/ef/4aab19d27fa8e58772be5c71c16add0426acf9e1f64353235c
Successfully built pickle5


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
import pickle5 as pickle

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Preprocessed_dataset_Nepal/english_data.csv')
print(df.head())

                                             content  category  \
0  Detroit: In explaining why Tesla Inc. keeps cu...  business   
1  Birgunj: Nepal has exported cement worth aroun...  business   
2  Kathmandu: The country has witnessed the inflo...  business   
3  Pokhara: Gold price has hit a new high in the ...  business   
4  Kathmandu: Nepal Oil Corporation (NOC) has red...  business   

                                                 url  \
0  https://www.eadarsha.com/eng/tesla-price-cuts-...   
1  https://www.eadarsha.com/eng/nepal-exports-cem...   
2  https://www.eadarsha.com/eng/remittance-inflow...   
3  https://www.eadarsha.com/eng/price-of-gold-set...   
4  https://www.eadarsha.com/eng/noc-slashed-price...   

                                               title                authors  \
0  Tesla Price Cuts: Knocking down EV rivals or t...                    RSS   
1  Nepal exports cement worth approx Rs 170 milli...  eAdarsha.com Reporter   
2  Remittance inflow increase

In [4]:
df.shape

(168011, 8)

In [5]:
df.columns

Index(['content', 'category', 'url', 'title', 'authors', 'published_date',
       'language', 'website_name'],
      dtype='object')

In [6]:
df = df.dropna(subset=['content'])

In [7]:
# Assuming 'content' is the column containing articles
df = df[['content']]
tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>')
tokenizer.fit_on_texts(df['content'])
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(df['content'])
padded_sequences = pad_sequences(sequences, maxlen=500, truncating='post')

In [8]:
# Save the tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [9]:
sequences = tokenizer.texts_to_sequences(df['content'])
padded_sequences = pad_sequences(sequences, maxlen=500, truncating='post')


In [10]:
# Simulate sentiment labels for training
np.random.seed(42)  # For reproducibility
df['simulated_sentiment'] = np.random.choice(['negative', 'neutral', 'positive'], size=len(df))

In [11]:
tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>')
tokenizer.fit_on_texts(df['content'])

In [12]:
sequences = tokenizer.texts_to_sequences(df['content'])
padded_sequences = pad_sequences(sequences, maxlen=500, truncating='post')

In [13]:
# Map simulated sentiment labels to numerical values
sentiment_mapping = {'positive': 2, 'neutral': 1, 'negative': 0}
df['sentiment_label'] = df['simulated_sentiment'].map(sentiment_mapping)
sentiment_labels = tf.keras.utils.to_categorical(df['sentiment_label'])

In [14]:
x_train, x_test, y_train, y_test = train_test_split(padded_sequences, sentiment_labels, test_size=0.2)

In [15]:
# Create a neural network with the specified layers
model = Sequential()
model.add(Embedding(5000, 500, input_length=500))
model.add(Conv1D(64, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 500)          2500000   
                                                                 
 conv1d (Conv1D)             (None, 496, 64)           160064    
                                                                 
 global_max_pooling1d (Globa  (None, 64)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 3)                 99        
                                                        

In [ ]:
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))


NameError: ignored

In [ ]:
# Save the sentiment analysis model
model.save('sentiment.h5')

In [ ]:
# Load the sentiment analysis model
model = tf.keras.models.load_model('sentiment.h5')

NameError: ignored

In [ ]:
# Perform sentiment analysis on the articles
sentiment_scores = model.predict(padded_sequences)
sentiment_labels = ['negative', 'neutral', 'positive']
df['sentiment_label'] = sentiment_scores.argmax(axis=1)
df['sentiment'] = df['sentiment_label'].apply(lambda x: sentiment_labels[x])
df['sentiment_score'] = sentiment_scores.max(axis=1)

NameError: ignored

In [ ]:
print(df.head())

NameError: ignored

The sentiment_label indicates the class with the highest predicted probability.
The sentiment_score for that class is a measure of how strong the model's confidence is in that prediction.

1. Sentiment Label (sentiment_label):
The sentiment_label represents the predicted sentiment class assigned to each article by the model. In our case, we have three sentiment classes: negative, neutral, and positive, represented by numerical values 0, 1, and 2 respectively. The sentiment_label is the model's best guess for the sentiment of the article based on its training. It is the class label with the highest predicted probability among the sentiment classes.

2. Sentiment Score (sentiment_score):
The sentiment_score is a probability value assigned by the model to each sentiment class for a given article. It indicates how confident the model is in its prediction. For example, if the sentiment_score for the "positive" class is high (close to 1), it means the model is very confident that the article has a positive sentiment. Conversely, if the score is low, the model is less confident in the prediction.

It's important to note that the actual numerical values assigned to sentiment_label and the predicted sentiment (positive, neutral, negative) might vary depending on how the model's classes were defined and the order in which they were encoded